In [1]:
from backfeed import *
import pandas as pd
import numpy as np

In [2]:
nas_path = "./dataset_1401/"
lineage_label = pd.read_csv('./dataset_1401/1404_lineage_report and metadata 20220316.csv')[['scorpio_call_y','diff']]
lineage_label = np.array(lineage_label.fillna("None"))
label_ = []
new_lineage_label = []
for idx, rna in enumerate(SeqIO.parse('./dataset_1401/1404.sequences.aln.fasta',"fasta")):
    # print(lineage_label[idx][0].split(' ')[0])
    label_.append(lineage_label[idx][0].split(' ')[0])

    new_lineage_label.append(str(rna.seq).replace('-','N'))

print('sample:', len(new_lineage_label))

sample: 1404


In [3]:
def clean(x):
	x = x.upper() 
	
	if x == 'T' or x == 'A' or x == 'G' or x == 'C' or x == '-' or x == 'N':
		return x

	if x == 'U' or x == 'Y':
		return 'T'
	
	if x == 'K' or x == 'S':
		return 'G'

	if x == 'M' or x == 'R' or x == 'W' or x == 'H' or x=='V' or x=='D':
		return 'A'

	if x== 'B':
		return 'C'

from tqdm.notebook import tqdm
dict_search = {'N':[1,0,0,0,0], 'A':[0,1,0,0,0], 'C':[0,0,1,0,0], 'G':[0,0,0,1,0], 'T':[0,0,0,0,1]}
total_sequence_array_onehot = []
for Seq in tqdm(new_lineage_label):
    temp_one_hot_Seq = []
    for nactg in Seq:
        unit = clean(nactg)
        temp_one_hot_Seq.append(dict_search[unit])
    total_sequence_array_onehot.append(np.array(temp_one_hot_Seq))
total_sequence_array_onehot = np.array(total_sequence_array_onehot)

  0%|          | 0/1404 [00:00<?, ?it/s]

In [4]:
total_sequence_array_onehot.shape

(1404, 29903, 5)

In [5]:
total_sequence_array_onehot[0][500:505], new_lineage_label[0][500:505]

(array([[0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1]]),
 'CACCT')

In [6]:
from pyDeepInsight import ImageTransformer, LogScaler
from tsnecuda import TSNE
from tqdm.notebook import tqdm

In [7]:
class_,_ ,_,_= np.unique(label_,return_counts=True,return_index=True,return_inverse=True)

In [8]:
class_dict_ = {}
for idx, i in enumerate(class_):
    class_dict_[i] = idx
print(class_dict_)
multi_label = []
for i in label_:
    multi_label.append(class_dict_[i])

{'Alpha': 0, 'B.1.1.318-like': 1, 'Beta': 2, 'Delta': 3, 'Eta': 4, 'Gamma': 5, 'Iota': 6, 'Lambda': 7, 'Mu': 8, 'None': 9}


In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=42)

train_indx, test_indx = next(sss.split(total_sequence_array_onehot, multi_label))
train_ids = [total_sequence_array_onehot[ind] for ind in train_indx]
train_labels = [multi_label[ind] for ind in train_indx]
print(len(train_ids), len(train_labels)) 

test_ids = [total_sequence_array_onehot[ind] for ind in test_indx]
test_labels = [multi_label[ind] for ind in test_indx]
print(len(test_ids), len(test_labels))

1053 1053
351 351


In [10]:
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import glob
from PIL import Image
import torch
import numpy as np
import random
np.random.seed(2020)
random.seed(2020)
torch.manual_seed(2020)

class TransferDataset(Dataset):
    def __init__(self, ids, labels, transform):
        self.transform = transform
        self.ids = ids
        self.labels = labels
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        singel_image_ = self.ids[idx].astype(np.float32)
        singel_image_ = singel_image_.flatten()
        seed = np.random.randint(1e9)       
        random.seed(seed)
        np.random.seed(seed)
        singel_image_ = torch.unsqueeze(torch.FloatTensor(singel_image_), 0)
        # singel_image_ = torch.unsqueeze(self.transform(singel_image_)[0], axis=0)
        label = int(self.labels[idx])

        return singel_image_, label

In [11]:
transformer = transforms.Compose([
            transforms.ToTensor(),
            # transforms.Normalize(mean, std),
            ])     

train_ds = TransferDataset(ids= train_ids, labels= train_labels, transform= transformer)
test_ds = TransferDataset(ids= test_ids, labels= test_labels, transform= transformer)
print(len(train_ds), len(test_ds))

1053 351


In [12]:
imgs, label = train_ds[10]
batch_size = 32
train_dl = DataLoader(train_ds, batch_size= batch_size, 
                        shuffle=True)
test_dl = DataLoader(test_ds, batch_size= 2*batch_size, 
                        shuffle=False)  

In [13]:
for xb,yb in train_dl:
    # print(xb.shape, yb.shape)
    # print(xb[0][0][0][0])
    print(yb[0])
    break

for xb,yb in test_dl:
    print(xb.shape, yb.shape)
    break

tensor(3)
torch.Size([64, 1, 149515]) torch.Size([64])


In [14]:
import torch.nn.functional as F
from torch import nn
class RNN(nn.Module):
    def __init__(self, class_n, Input_Size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=Input_Size,
            hidden_size=128,
            num_layers=1,
            batch_first=True,
        )

        self.out = nn.Linear(128, class_n)
    def forward(self, x):
        r_out, (h_c, h_h) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = RNN(len(class_), 149515).to(device)
model.initialize_weights()
# model.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
# model

In [15]:
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# loss_func = nn.CrossEntropyLoss(reduction="sum", weight=class_weights)
loss_func = nn.CrossEntropyLoss(reduction="sum")
opt = optim.Adam(model.parameters(), lr=0.003)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=5,verbose=1)
os.makedirs("./models", exist_ok=True)
path2weights = "./models/weights_LSTM_multiclass(onehot).2022.03.22.pt"
params_train={
    "num_epochs": 100,
    "optimizer": opt,
    "loss_func": loss_func,
    "train_dl": train_dl,
    "val_dl": test_dl,
    "sanity_check": False,
    "lr_scheduler": lr_scheduler,
    "path2weights": path2weights,
    }

In [16]:
import copy
from tqdm import tqdm_notebook
def loss_epoch(model,loss_func,dataset_dl,sanity_check=False,opt=None):
    running_loss=0.0
    running_metric=0.0
    len_data = len(dataset_dl.dataset)
    for xb, yb in tqdm_notebook(dataset_dl):
    # for xb, yb in (dataset_dl):
        xb=xb.to(device)
        yb=yb.to(device)
        # print(type(xb), type(yb.shape))
        output=model(xb)
        loss_b,metric_b=loss_batch(loss_func, output, yb, opt)
        running_loss+=loss_b
        
        if metric_b is not None:
            running_metric+=metric_b
        if sanity_check is True:
            break
    loss=running_loss/float(len_data)
    metric=running_metric/float(len_data)
    return loss, metric

def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

def metrics_batch(output, target):
    pred = output.argmax(dim=1, keepdim=True)
    corrects=pred.eq(target.view_as(pred)).sum().item()
    return corrects

def loss_batch(loss_func, output, target, opt=None):
    loss = loss_func(output, target)
    with torch.no_grad():
        metric_b = metrics_batch(output,target)
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
    return loss.item(), metric_b

def train_val(model, params):
    num_epochs=params["num_epochs"]
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    val_dl=params["val_dl"]
    sanity_check=params["sanity_check"]
    lr_scheduler=params["lr_scheduler"]
    path2weights=params["path2weights"]
    
    loss_history={
        "train": [],
        "val": [],
    }
    
    metric_history={
        "train": [],
        "val": [],
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss=float('inf')
    
    for epoch in range(num_epochs):
        current_lr=get_lr(opt)
        print('Epoch {}/{}, current lr={}'.format(epoch, num_epochs - 1, current_lr))
        model.train()
        train_loss, train_metric=loss_epoch(model,loss_func,train_dl,sanity_check,opt)
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        model.eval()
        with torch.no_grad():
            val_loss, val_metric=loss_epoch(model,loss_func,val_dl,sanity_check)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), path2weights)
            print("Copied best model weights!")
        
        loss_history["val"].append(val_loss)
        metric_history["val"].append(val_metric)
        
        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            print("Loading best model weights!")
            model.load_state_dict(best_model_wts)
        

        print("train loss: %.6f, dev loss: %.6f,  train accuracy: %.2f,valid accuracy: %.2f" %(train_loss,val_loss, 100*train_metric,100*val_metric))
        print("-"*10) 
    model.load_state_dict(best_model_wts)
        
    return model, loss_history, metric_history

In [17]:
model,loss_hist,metric_hist = train_val(model,params_train)

Epoch 0/99, current lr=0.003


/home/john/anaconda3/envs/deepinsightlearning/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Copied best model weights!
train loss: 0.989104, dev loss: 0.710750,  train accuracy: 82.15,valid accuracy: 84.33
----------
Epoch 1/99, current lr=0.003


...

train loss: 0.688643, dev loss: 0.693545,  train accuracy: 84.33,valid accuracy: 84.33
----------
Epoch 32/99, current lr=0.0001875


  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

train loss: 0.688631, dev loss: 0.693604,  train accuracy: 84.33,valid accuracy: 84.33
----------
Epoch 33/99, current lr=0.0001875


  0%|          | 0/33 [00:00<?, ?it/s]